# Загружаем изображение

In [1]:
import numpy as np
import cv2

# Load an color image in grayscale
img_ld = cv2.imread('cog.png',0)
_,img = cv2.threshold(img_ld, 127, 255, 0)
_,image_stock = cv2.threshold(img_ld, 127, 255, 0)

In [2]:
img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [29]:
cv2.imshow('image',image_stock)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 1 операция - эрозия исх. изображения элементом hole-ring

In [4]:
dim_1 = 99
dim_2 = 97

kernel_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_1,dim_1))
kernel_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_2,dim_2))
addin_1 = np.zeros((dim_2,1))
kernel_2 = np.hstack((addin_1,kernel_2,addin_1))
addin_2 = np.zeros((1,dim_1))
kernel_2 = np.vstack((addin_2,kernel_2,addin_2))
kernel_2
kernel_ring = kernel_1 - kernel_2
kernel_ring = kernel_ring.astype('uint8')

In [5]:
image_eroded = cv2.erode(image_stock,kernel_ring)

In [7]:
cv2.imshow('image',image_eroded)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2 Операция - dilute того, что получилось 8-угольником

In [8]:
dim_1 = 99
dim_2 = int(dim_1/3)
kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT,(dim_1,dim_1))
for i in range(dim_2+1):
    for j in range(dim_2+1):
        kernel_1[i][j] = 0
        kernel_1[dim_1-i-1][j] = 0
        kernel_1[i,dim_1-j-1] = 0
        kernel_1[dim_1-i-1][dim_1-j-1] = 0
kernel_octagon = kernel_1

In [9]:
cv2.imshow('image',kernel_octagon)
cv2.waitKey(0)
cv2.destroyAllWindows()
kernel_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_1+2,dim_1+2))

In [10]:
image_dilated = cv2.dilate(image_eroded,kernel_1)

In [11]:
cv2.imshow('image',image_dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 3 Операция - сложение

In [12]:
image_added = image_stock|image_dilated

In [13]:
cv2.imshow('image',image_added)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 4 Операция - размыкание на Gear_Body

In [16]:
dim_1 = 285
#kernel_1 = np.zeros((dim_1,dim_1))
#cv2.circle(kernel_1,(int(dim_1/2),int(dim_1/2)),int(dim_1/2),255)
#kernel_1 = kernel_1.astype('uint8')
kernel_gear_body = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_1,dim_1))
#image_closed = cv2.morphologyEx(image_added, cv2.MORPH_OPEN, kernel_1)
image_closed = cv2.morphologyEx(image_added, cv2.MORPH_OPEN, kernel_gear_body)

In [17]:
cv2.imshow('image',image_closed)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
dim_2 = 20
kernel_sampling_ring_spacer = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_2,dim_2))

In [19]:
image_tmp = cv2.dilate(image_closed,kernel_sampling_ring_spacer)

In [20]:
cv2.imshow('image',image_tmp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
image_sel = image_tmp-image_closed

In [22]:
cv2.imshow('image',image_sel)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
dim_2 = 302
kernel = np.zeros((dim_2,dim_2))
cv2.circle(kernel,(int(dim_2/2),int(dim_2/2)),int(dim_2/2),255)
kernel = kernel.astype('uint8')
image_inflated = cv2.morphologyEx(image_sel, cv2.MORPH_OPEN, kernel)

In [24]:
cv2.imshow('image',image_inflated)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 5 Операция - логическое умножение на исходное изображение

In [25]:
image_mult_src = image_inflated & image_stock

In [26]:
cv2.imshow('image',image_mult_src)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 6 Операция - выделяем зубцы

In [27]:
dim_3 = 20
kernel_teeth = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_3,dim_3))
image_cogteeth = cv2.dilate(image_mult_src,kernel_teeth)

In [30]:
cv2.imshow('image',image_cogteeth)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
image_defects = image_sel - image_cogteeth
cv2.imshow('image',image_defects)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
dim_4 = 40
kernel_defects = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(dim_4,dim_4))
image_defects_infl = cv2.dilate(image_defects,kernel_defects)
cv2.imshow('image',image_defects_infl)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image_defects_final = image_defects_infl | image_cogteeth
cv2.imshow('image',image_defects_final)
cv2.waitKey(0)
cv2.destroyAllWindows()